In [1]:
import cv2
import sqlite3
from matplotlib import pyplot as plt
import numpy as np

from visualize_model import Model
from database import blob_to_array, pair_id_to_image_ids

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. 3D mesh reconstruction from a set of images from the Gerrard Hall dataset.
Install Colmap and run the automatic reconstruction on the Gerrard Hall dataset

1) We downloaded the `gerrard-hall` dataset from https://colmap.github.io/datasets.html and stored it on path `lab4/gerrard-hall`.

2) We installed Colmap on UNIX really easily with any package manager of choice:

```bash
$ brew install colmap
$ colmap -h # to verify that works
```

3) Then we ran the `automatic_reconstructor` pipeline (as seen in their docs [here](https://github.com/colmap/colmap.github.io/blob/master/_sources/cli.rst.txt)) via CLI.

```bash
$ cd lab4/
$ colmap automatic_reconstructor \
    --workspace_path . \
    --image_path ./gerrard-hall/images
```

The reconstruction is stored under the `lab4/database.db` path.

\* Note that if we want to try different parameters we will have to run the reconstruction in different steps (reference [here](https://github.com/colmap/colmap.github.io/blob/master/_sources/cli.rst.txt))


# 2. Analyze reconstructions using python
## 2.1. Run the notebook, using the Gerrard Hall reconstruction (0.5)
#### <span style='color:Green'> - Add the path to your reconstruction. Answer the questions at the end  </span>

In [2]:
# Add your path
reconstruction_path = "./sparse/0/"
database_path = "./database.db"

#### Load an existing reconstruction and print its contents

In [3]:
model = Model()
model.read_model(reconstruction_path, ext='.bin') # Should also work with .txt

In [4]:
images = model.images
cameras = model.cameras
points3D = model.points3D

In [5]:
"""
from COLMAP docs: https://colmap.github.io/format.html 
The unique identifiers of cameras (CAMERA_ID), images (IMAGE_ID), and 3D points (POINT3D_ID) 
are unordered and are most likely not contiguous. 
"""

idx_images = next(iter(images))
idx_camera = next(iter(cameras))
idx_point3d = next(iter(points3D))

print(f"Loaded {len(images)} images. This is the information available for one of them:")
print(images[idx_images])
print(f"\nLoaded {len(cameras)} cameras. This is the information available for one of them:")
print(cameras[idx_camera])
print(f"\nLoaded {len(points3D)} 3D points. This is the information available for one of them:")
print(points3D[idx_point3d])

Loaded 98 images. This is the information available for one of them:
Image(id=1, qvec=array([-0.13316643,  0.01938725,  0.97132254, -0.19601879]), tvec=array([0.94717796, 1.08745837, 4.4189751 ]), camera_id=1, name='IMG_2331.JPG', xys=array([[1729.12634277, 1121.50134277],
       [ 874.54071045, 2874.25634766],
       [4424.20898438, 2655.8605957 ],
       ...,
       [5564.17236328,  256.39727783],
       [1752.71594238,  257.99530029],
       [ 823.69580078,  260.04330444]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1]))

Loaded 1 cameras. This is the information available for one of them:
Camera(id=1, model='SIMPLE_RADIAL', width=5616, height=3744, params=array([ 3.82402417e+03,  2.80800000e+03,  1.87200000e+03, -6.97168319e-02]))

Loaded 71273 3D points. This is the information available for one of them:
Point3D(id=96251, xyz=array([ 2.28315334, -0.64964237, -1.2782611 ]), rgb=array([67, 67, 68]), error=array(2.55957303), image_ids=array([71, 72, 83, 69, 76, 73]), point2D_idxs=

#### Load the database

In [6]:
db = sqlite3.connect(database_path)

In [7]:
keypoints = dict(
        (image_id, blob_to_array(data, np.float32, (-1, 2)))
        for image_id, data in db.execute(
            "SELECT image_id, data FROM keypoints"))

In [8]:
print(f"Loaded keypoints from {len(keypoints)} images. These are the {len(keypoints[1])} keypoints for one of them:")
print(keypoints[1])

Loaded keypoints from 100 images. These are the 32652 keypoints for one of them:
[[ 1.7291263e+03  1.1215013e+03]
 [ 3.9597284e+02  0.0000000e+00]
 [ 2.4773894e+02  6.4793530e+02]
 ...
 [ 8.2369580e+02  2.6004330e+02]
 [ 4.0847859e+00 -1.0391687e-15]
 [ 4.4129825e-01  5.0973716e+00]]


In [9]:
matches = dict()
count_no_data = 0
for pair_id, data in db.execute("SELECT pair_id, data FROM matches"):
    if data is None:
        count_no_data += 1
    else:
        matches[pair_id_to_image_ids(pair_id)] = blob_to_array(data, np.uint32, (-1, 2))
print(f"Loaded {len(matches)} matches. {count_no_data}/{len(matches)+count_no_data} matches contained no data")

Loaded 1879 matches. 3071/4950 matches contained no data


In [10]:
print("These are the matches between two images:")
print(matches[1,2])

These are the matches between two images:
[[    0     0]
 [    1     2]
 [    3     6]
 ...
 [10506 10705]
 [ 9586 10799]
 [ 9568 10852]]


#### Visualize the point cloud and cameras

In [11]:
model.create_window()
model.add_points()
model.add_cameras(scale=0.25)
model.show()

#### <span style='color:Green'>  How many keypoints there are in total? </span> 

In [12]:
# show tables
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = tables.fetchall()

# Enumerate and print the names of the tables
for i, table in enumerate(tables):
    print(f"Table {i+1}: {table[0]}")


Table 1: cameras
Table 2: sqlite_sequence
Table 3: images
Table 4: keypoints
Table 5: descriptors
Table 6: matches
Table 7: two_view_geometries


#### <span style='color:Green'>  How many 3D points originated from a keypoint in the first image? </span>


## 2.2 Plot the 3D points coloured according to the number of images and error. (0.5)

#### <span style='color:Green'> - Plot the 3D points coloured according to the **number of images** from which it originated. </span> Can you extract any conclusions from the visualization? 

In [ ]:
### TO DO 2.2

#### <span style='color:Green'> - Plot the 3D points coloured according to the **error**. </span> - What is this parameter? Can you extract any conclusions from the visualization?

In [ ]:
### TO DO 2.2

## 2.3 Plot the 3D points that correspond to a keypoint in the first image. Also plot the image with the keypoints (1.0)


In [ ]:
### TO DO 2.3

## 2.4 Create a visualization for the number of matches between all images. (1.0)
For example: https://seaborn.pydata.org/generated/seaborn.heatmap.html

In [ ]:
### TO DO 2.4

## 2.5 Visualize the keypoints and matches between the two images used in lab 3 using Colmap, how it compares to the results from lab 3? (1.0)
#### <span style='color:Green'> You can use the GUI to get the keypoints and matches and then visualize it here, following the same style as in lab 3 to get comparable results. </span>

In [ ]:
### TO DO 2.5

## 2.6 Triangulate and visualize the 3D points from the keypoints extracted using Colmap on the two images used in lab 3, how it compares to the results from lab 3? (1.0) 
#### <span style='color:Green'> - Use the triangulation from lab 3 to the get the 3D points and visualize them following the same style. </span>

In [ ]:
### TO DO 2.6

## 2.7 Visualize the sparse reconstruction using the 2 images from lab 3, and the complete CASTLE dataset. Comment on the differences between techniques and number of images used. (1.0)
#### <span style='color:Green'> - Use the reconstruction from Colmap to the get the 3D points and visualize them following the same style, using two images and the complete dataset. </span>

In [ ]:
### TO DO 2.7